In [8]:
# CUDA 12.1
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

!pip install torch_geometric
!pip install torch-geometric-temporal
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install networkx numpy==1.24.3


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 4.2 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/12.9 MB 9.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.9 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 15.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.9 MB 8.4 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.9 MB 13.4 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.9 MB 14.4 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.9 MB 14.2 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 14.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.8/14.8 MB 8.4 MB/s eta 0:00:02
     -------------- ------------------------- 5.5/14.8 MB 12.9 MB/s eta 0:00:01
     ---------------------- ----------------- 8.4/14.8 MB 14.1 MB/s eta 0:00:01
     ------------------------------- ------- 12.1/14.8 MB 14.5 MB/s eta 0:00:01
     --------------------------------------- 14.8/14.8 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
import random

# 可复现
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# ==== 1. 生成离散时间动态图 ====
def generate_dynamic_graph(num_nodes=10, num_timesteps=6):
    edge_indices = []
    edge_weights = []
    features = []
    targets = []

    for t in range(num_timesteps):
        # 固定节点集合，防止 node index 越界
        g = nx.erdos_renyi_graph(num_nodes, p=0.3, seed=t)
        g.add_nodes_from(range(num_nodes))  # 确保节点为 0..num_nodes-1

        edges = list(g.edges())
        if len(edges) == 0:
            edge_index = np.empty((2, 0), dtype=np.int64)
            edge_attr = np.empty((0, 2), dtype=np.float32)
        else:
            edge_index = np.array(edges, dtype=np.int64).T
            edge_attr = np.random.rand(edge_index.shape[1], 2).astype(np.float32)

        # 每个时间步的节点特征
        x = np.random.rand(num_nodes, 3).astype(np.float32)

        # 边标签（示例规则）
        if edge_attr.shape[0] == 0:
            y = np.empty((0,), dtype=np.int64)
        else:
            y = (edge_attr.sum(axis=1) > 1.0).astype(np.int64)

        w = edge_attr[:, 0].astype(np.float32) if edge_attr.shape[0] > 0 else np.empty((0,), dtype=np.float32)

        edge_indices.append(edge_index)
        edge_weights.append(w)
        features.append(x)
        targets.append(y)

    dataset = DynamicGraphTemporalSignal(
        edge_indices=edge_indices,
        edge_weights=edge_weights,
        features=features,
        targets=targets
    )
    return dataset

dataset = generate_dynamic_graph(num_nodes=10, num_timesteps=8)
print(f"Snapshots: {dataset.snapshot_count}")

# ==== 2. 模型 ====
class RecurrentLinkClassifier(torch.nn.Module):
    def __init__(self, node_features, hidden_dim):
        super().__init__()
        self.recurrent = GConvGRU(node_features, hidden_dim, K=2)
        self.fc = torch.nn.Linear(hidden_dim * 2, 2)  # binary logits

    def forward(self, x, edge_index, h):
        h = self.recurrent(x, edge_index, h)
        return h

    def predict_links(self, h, edge_index):
        src, dst = edge_index
        # 确保索引不越界
        if src.max() >= h.size(0) or dst.max() >= h.size(0):
            raise ValueError(f"edge_index out of range: node count={h.size(0)}, max src={src.max()}, max dst={dst.max()}")
        z = torch.cat([h[src], h[dst]], dim=1)
        return self.fc(z)

# ==== 3. 训练 ====
device = torch.device("cpu")
model = RecurrentLinkClassifier(node_features=3, hidden_dim=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def to_tensor(data, dtype=torch.float32):
    if isinstance(data, np.ndarray):
        return torch.from_numpy(data).to(dtype=dtype, device=device)
    elif isinstance(data, torch.Tensor):
        return data.to(dtype=dtype, device=device)
    else:
        raise TypeError(f"Unexpected data type: {type(data)}")

epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    h = None

    for t, snapshot in enumerate(dataset):
        x = to_tensor(snapshot.x, torch.float32)
        edge_index = to_tensor(snapshot.edge_index, torch.long)
        if edge_index.shape[1] == 0:
            continue

        y_data = snapshot.y
        y = torch.from_numpy(y_data).long().to(device) if isinstance(y_data, np.ndarray) else y_data.long().to(device)

        h = model(x, edge_index, h)
        preds = model.predict_links(h, edge_index)

        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += float(loss.item())

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss:.4f}")

# ==== 4. 评估 ====
model.eval()
h = None
correct, total = 0, 0
with torch.no_grad():
    for snapshot in dataset:
        x = to_tensor(snapshot.x, torch.float32)
        edge_index = to_tensor(snapshot.edge_index, torch.long)
        if edge_index.shape[1] == 0:
            continue

        y_data = snapshot.y
        y = torch.from_numpy(y_data).long().to(device) if isinstance(y_data, np.ndarray) else y_data.long().to(device)

        h = model(x, edge_index, h)
        preds = model.predict_links(h, edge_index)
        pred_labels = preds.argmax(dim=1)
        correct += (pred_labels == y).sum().item()
        total += y.size(0)

if total > 0:
    print(f"Accuracy: {correct / total:.4f} ({correct}/{total})")
else:
    print("No edges to evaluate.")


Snapshots: 8
